<a href="https://colab.research.google.com/github/Yash0411/Traffic-Sign-Detection-/blob/master/Traffic_sign_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

In [0]:
from google.colab import files
files.upload() #upload kaggle.json
 
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
kaggle.json


In [0]:
!kaggle datasets download -d eunjurho/german-traffic-sign-recognition-benchmark-cropped

 97% 200M/206M [00:02<00:00, 108MB/s] 
100% 206M/206M [00:02<00:00, 85.6MB/s]


In [0]:
!unzip -q /content/german-traffic-sign-recognition-benchmark-cropped.zip -d .
!ls

german-traffic-sign-recognition-benchmark-cropped.zip  kaggle.json
gtsrb-preprocessed				       sample_data


In [0]:
import os

training_dir = os.listdir('/content/gtsrb-preprocessed/train')
testing_dir = os.listdir('/content/gtsrb-preprocessed/test')

print(len(training_dir))
print(len(testing_dir))


43
12630


In [0]:
os.mkdir("/content/train/")
os.mkdir("/content/test/")

In [0]:
for i in training_dir:
  os.mkdir("/content/train/"+i)
  os.mkdir("/content/test/"+i)

In [0]:
import random
from shutil import copyfile

def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    
    files = []
    
    for filename in os.listdir(SOURCE):
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + "is zero so ignoring length")
            
    
    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) * (1 - SPLIT_SIZE))
    shuffeled_set = random.sample (files, len(files))
    training_set = shuffeled_set[0 : training_length]
    testing_set = shuffeled_set[~testing_length : ]
    
    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)
        
    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination) 


for i in training_dir:
  split_data("/content/gtsrb-preprocessed/train/"+i+"/", "/content/train/"+i+"/", "/content/test/"+i+"/", 0.96)

  print(len(os.listdir('/content/train/'+i)))
  print(len(os.listdir('/content/test/'+i)))

NameError: ignored

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

classifier = Sequential()


classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Conv2D(64, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Conv2D(64, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Flatten())

classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units = 43, activation = 'softmax'))

classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
TRAINING_DIR = "/content/train"
train_datagen = ImageDataGenerator( rescale=1./255.)

training_set = train_datagen.flow_from_directory(TRAINING_DIR,
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')




VALIDATION_DIR = "/content/test"
validation_datagen = ImageDataGenerator(rescale = 1.0 / 255)

test_set = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [0]:
history = classifier.fit_generator(training_set,
                         epochs = 32,
                         validation_data = test_set)

In [0]:
# PLOT LOSS AND ACCURACY
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')


In [0]:
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(64,64))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  image = np.vstack([x])
  classes = classifier.predict(image)
  print(classes)